In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [7]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"melinda29","key":"5c3540edbbe023c305b511f5f39a6d56"}'}

In [8]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [9]:
!kaggle datasets download -d fedesoriano/heart-failure-prediction

Dataset URL: https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction
License(s): ODbL-1.0
  0% 0.00/8.56k [00:00<?, ?B/s]
100% 8.56k/8.56k [00:00<00:00, 17.2MB/s]


In [10]:
!mkdir heart
!unzip heart-failure-prediction.zip -d heart
!ls heart

Archive:  heart-failure-prediction.zip
  inflating: heart/heart.csv         
heart.csv


In [11]:
df = pd.read_csv('/content/heart/heart.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [13]:
df.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [14]:
overall_duplicates = df[df.duplicated()]
print("Duplikat")
print(overall_duplicates)

Duplikat
Empty DataFrame
Columns: [Age, Sex, ChestPainType, RestingBP, Cholesterol, FastingBS, RestingECG, MaxHR, ExerciseAngina, Oldpeak, ST_Slope, HeartDisease]
Index: []


In [15]:
df.drop_duplicates(inplace=True)

print("DataFrame setelah menghapus duplikat:")
print(df)

DataFrame setelah menghapus duplikat:
     Age Sex ChestPainType  RestingBP  Cholesterol  FastingBS RestingECG  \
0     40   M           ATA        140          289          0     Normal   
1     49   F           NAP        160          180          0     Normal   
2     37   M           ATA        130          283          0         ST   
3     48   F           ASY        138          214          0     Normal   
4     54   M           NAP        150          195          0     Normal   
..   ...  ..           ...        ...          ...        ...        ...   
913   45   M            TA        110          264          0     Normal   
914   68   M           ASY        144          193          1     Normal   
915   57   M           ASY        130          131          0     Normal   
916   57   F           ATA        130          236          0        LVH   
917   38   M           NAP        138          175          0     Normal   

     MaxHR ExerciseAngina  Oldpeak ST_Slope  Hear

In [16]:
df = df.reset_index(drop=True)

In [17]:
numerical = []
catgcols =[]

for col in df.columns:
    if df[col].dtype=="float64":
      numerical.append(col)
    else:
        catgcols.append(col)

In [18]:
for col in df.columns:
  if col in numerical:
    df[col].fillna(df[col].median(), inplace=True)
  else:
    df[col].fillna(df[col].mode()[0], inplace=True)

In [19]:
le = LabelEncoder()

for col in catgcols:
  df[col] = le.fit_transform(df[col])

In [20]:
ind_col = [col for col in df.columns if col != 'HeartDisease']
dep_col = 'HeartDisease'

In [21]:
df['HeartDisease'].value_counts()

HeartDisease
1    508
0    410
Name: count, dtype: int64

In [22]:
x = df[ind_col]
y = df[dep_col]

In [23]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=80, stratify=y)

print('train shape:', x_train.shape)
print('test shape:', x_test.shape)

train shape: (642, 11)
test shape: (276, 11)


In [24]:
model = Sequential()

model.add(Dense(8, activation='relu', input_dim=x.shape[-1]))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 96        
                                                                 
 dense_1 (Dense)             (None, 16)                144       
                                                                 
 dense_2 (Dense)             (None, 2)                 34        
                                                                 
Total params: 274 (1.07 KB)
Trainable params: 274 (1.07 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
optim = Adam(lr=0.001)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optim,
              metrics=['acc'])


batch_size=1
epochs = 15


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    shuffle=True,
                    validation_split=0.1)

Epoch 1/15
577/577 [==============================] - 2s 2ms/step - loss: 0.7642 - acc: 0.5841 - val_loss: 0.5498 - val_acc: 0.7692
Epoch 2/15
577/577 [==============================] - 1s 2ms/step - loss: 0.6288 - acc: 0.6759 - val_loss: 0.5062 - val_acc: 0.8000
Epoch 3/15
577/577 [==============================] - 1s 2ms/step - loss: 0.5845 - acc: 0.7158 - val_loss: 0.4129 - val_acc: 0.8154
Epoch 4/15
577/577 [==============================] - 1s 2ms/step - loss: 0.5650 - acc: 0.7435 - val_loss: 0.4728 - val_acc: 0.8154
Epoch 5/15
577/577 [==============================] - 2s 3ms/step - loss: 0.5246 - acc: 0.7643 - val_loss: 0.3955 - val_acc: 0.8462
Epoch 6/15
577/577 [==============================] - 1s 3ms/step - loss: 0.5105 - acc: 0.7730 - val_loss: 0.3618 - val_acc: 0.8308
Epoch 7/15
577/577 [==============================] - 1s 2ms/step - loss: 0.4896 - acc: 0.7903 - val_loss: 0.3352 - val_acc: 0.8615
Epoch 8/15
577/577 [==============================] - 1s 2ms/step - loss: 0.

In [27]:
pred = np.argmax(model.predict(x_test), axis=1)
print(accuracy_score(y_test, pred))

9/9 [==============================] - 0s 3ms/step
0.8188405797101449


In [28]:
prediction = model.predict([[12, 1,	1,	41,	147,	0,	1,	98,	0,	0.0,	2]])
print(prediction)

1/1 [==============================] - 0s 120ms/step
[[0.8228494  0.17715059]]


In [29]:
prediction = model.predict([[9,	1, 1,	31,	141,	0,	2,	25,	0,	0.0,	2	]])

predicted_class = np.argmax(prediction, axis=-1)

if predicted_class == 1:
    print('Pasien Terkena Gagal Jantung')
else:
    print('Pasien Normal')

1/1 [==============================] - 0s 63ms/step
Pasien Terkena Gagal Jantung


In [30]:
prediction = model.predict([[20,	0,	0,	39,	72,	0,	1,	34,	1,	1.5,	1	]])

predicted_class = np.argmax(prediction, axis=-1)

if predicted_class == 1:
    print('Pasien Terkena Gagal Jantung')
else:
    print('Pasien Normal')

1/1 [==============================] - 0s 82ms/step
Pasien Terkena Gagal Jantung


In [31]:
model.save('./model.hs')

In [32]:
model = tf.keras.models.load_model('./model.hs')
converter =tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
open("./model.tflite", "wb").write(tflite_model)

3192